In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit_aer import AerSimulator

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive inside a session. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# with Session(backend=service.backend("ibmq_qasm_simulator")):
#     result = Sampler().run(circuits).result()

In [2]:
from math import pi
from qiskit.providers.ibmq import *
from qiskit import IBMQ

In [3]:
def prepare_state(bit_string : str):
    """Applies PauliX gates to qubits wherever there is a computational 1 state in the input"""
    bit_string = bit_string[::-1]
    n = len(bit_string)
    qc = QuantumCircuit(n)
    for i, s in zip(range(n), bit_string):
        if s == '1':
            qc.x(i)
    return qc

def equalize_lengths(str1, str2):
    """Pads zeroes to the most significant bit of the number with lesser number of bits"""
    p,q = len(str1), len(str2)
    
    if p>q:
        str2 = '0'*(p-q) + str2
    elif q>p:
        str1 = '0'*(q-p) + str1
        
    return str1, str2

# QFT from scratch
def qft_rotations(circuit, n:int):
    """Performs qft on the first n qubits in circuit (without swaps)"""
    if n == 0:
        return circuit
    n -= 1
    circuit.h(n)
    for qubit in range(n):
        circuit.cp(pi/2**(n-qubit), qubit, n)
    # At the end of our function, we call the same function again on
    # the next qubits (we reduced n by one earlier in the function)
    qft_rotations(circuit, n)

def swap_registers(circuit, n:int):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit

def qft(circuit, n:int, do_swaps:bool=True):
    """QFT on the first n qubits in circuit"""
    qft_rotations(circuit, n)
    if do_swaps:
        swap_registers(circuit, n)
    return circuit

def inverse_qft(circuit, n:int, do_swaps:bool=True):
    """Does the inverse QFT on the first n qubits in circuit"""
    if do_swaps:
        # First we create a QFT circuit of the correct size:
        qft_circ = qft(QuantumCircuit(n), n)
        # Then we take the inverse of this circuit
        invqft_circ = qft_circ.inverse()
    else:
        qft_circ = qft(QuantumCircuit(n), n, do_swaps=False)
        invqft_circ = qft_circ.inverse()
    # And add it to the first n qubits in our existing circuit
    circuit.append(invqft_circ, circuit.qubits[:n])
    return circuit
    #return circuit.decompose() # .decompose() allows us to see the individual gates

In [4]:
def QFT_ADD(num1:str, num2:str):
    """Using the QFT that we built above to build a circuit that stores the sum of two numbers in the output register"""
    
    p,q = len(num1), len(num2)
    
    if p!=q:
        num1, num2 = equalize_lengths(num1, num2)
    n = len(num1)    
    
    qft_circ_1 = QuantumCircuit(2*n+1,n+1)

    qft_circ_1.append(prepare_state(num1), list(range(n)))
    qft_circ_1.append(prepare_state(num2), list(range(n,2*n)))

    qft_circ_1.barrier()

    qft_circ_1.append(qft(QuantumCircuit(n+1), n+1, do_swaps=False), list(range(n,2*n+1)))

    qft_circ_1.barrier()

    for k in range(n):
        theta_1 = 2*pi / 2**(k+2)
        qft_circ_1.cp(theta_1, control_qubit=n-k-1, target_qubit=2*n)

    for i in range(n):
        for j in range(n-i):
            theta = 2*pi / 2**(j+1)
            qft_circ_1.cp(theta, control_qubit=n-i-j-1, target_qubit=2*n-i-1)

    qft_circ_1.barrier()

    qft_circ_1.append(inverse_qft(QuantumCircuit(n+1), n+1, do_swaps=False), list(range(n,2*n+1)))

    qft_circ_1.barrier()

    qft_circ_1.measure(list(range(n,2*n+1)), list(range(n+1)))
    
    return qft_circ_1

In [5]:
def qft_mult_rotations(n):
    
    q_input1 = QuantumRegister(n, name='input1')
    q_input2 = QuantumRegister(n, name='input2')
    q_output = QuantumRegister(2*n, name='output')
    
    qc = QuantumCircuit(q_input1, q_input2, q_output)

    for j in range(1,n+1):
        for i in range(1, n+1):
            for k in range(1,2*n+1):
                phase = 2*pi / 2**(i+j+k-2*n)
                gate = PhaseGate(phase).control(2)
                qc.append(gate, [q_input1[n-j], q_input2[n-i], q_output[k-1]])
                
    return qc

In [6]:
def QFT_MULTIPLY(num1:str, num2:str):
    """Using the QFT that we built above to build a circuit that stores the product of two numbers in the output register"""
    
    if len(num1) != len(num2):
        num1, num2 = equalize_lengths(num1, num2)
    n = len(num1)

    q1 = QuantumRegister(n, name='input1')
    q2 = QuantumRegister(n, name='input2')
    q3 = QuantumRegister(2*n, name='output')
    c = ClassicalRegister(2*n, name='meas')
    qc = QuantumCircuit(q1,q2,q3,c)

    qc.append(prepare_state(num1), list(range(n)))
    qc.append(prepare_state(num2), list(range(n,2*n)))

    qc.barrier()
    qc.append(qft(QuantumCircuit(2*n), 2*n, do_swaps=False), list(range(2*n, 4*n)))
    qc.barrier()

    qc.append(qft_mult_rotations(n), list(range(4*n)))

    qc.barrier()
    qc.append(inverse_qft(QuantumCircuit(2*n), 2*n, do_swaps=False), list(range(2*n, 4*n)))
    qc.barrier()

    qc.measure(list(range(2*n, 4*n)), list(range(2*n)))
    
    return qc